In [1]:
#Load modules
import zarr
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import yaml
from pathlib import Path
import allel

from dask.distributed import Client
import dask
dask.config.set(**{'array.slicing.split_large_chunks': False}) # Silence large chunk warnings
import dask.array as da
from dask import delayed, compute
from dask_gateway import Gateway
import functools
import numcodecs
from fsspec.implementations.zip import ZipFileSystem
from collections.abc import Mapping
import gcsfs
import numba
import psutil
from humanize import naturalsize

import pickle
import platform

import traceback
import logging

from pyprojroot import here
from bokeh.plotting import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator

### Connect to gcs

In [2]:
gcs = gcsfs.GCSFileSystem()

/home/conda/global/60d1d102327c6b963c52da1c248570d56dc2ebf066625a421ba924078e5d2fa7-20230228-170115-620472-19-binder-4.3.0/lib/python3.10/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
gcs.ls('vo_afun_release')[:3]

['vo_afun_release/reference',
 'vo_afun_release/v1.0',
 'vo_afun_release/v1.0-config.json']

### Set up data access

In [4]:
production_root = Path('vo_afun_release')
vo_afun_staging = Path(production_root, 'v1.0')
sampleset_staging_dir = Path(vo_afun_staging, 'snp_genotypes', 'all')

#Decision tree or static filters
genomic_positions_site_filter_dt_data_cloud_zarr_dir = 'vo_afun_release/v1.0/site_filters/dt_20200416/funestus'
genomic_positions_site_filter_sc_data_cloud_zarr_dir = 'vo_afun_release/v1.0/site_filters/sc_20220908/funestus'

repo_clone_path = here()
release_config_path = repo_clone_path / 'tracking' / 'release' / 'v1.0' / 'config.yml'

with open(release_config_path) as fh:
    config = yaml.load(fh, Loader=yaml.BaseLoader)
    
samplesets = config["sample_sets"]
#Remove the sample set with no sample
samplesets.remove("1242-VO-KE-TCHOUASSI-VMF00082")

In [5]:
meta = pd.read_csv("../../metadata/supp1.csv")
meta.columns

Index(['sample_id', 'geographic_cohort', 'geographic_cohort_colour',
       'geographic_cohort_shape', 'PCA_cohort', 'PCA_cohort_colour',
       'mitochondrial_id', 'karyotype_3La', 'karyotype_3Ra', 'karyotype_3Rb',
       'karyotype_2Ra', 'karyotype_2Rh', 'median_coverage'],
      dtype='object')

### Functions

In [6]:
# load a single array from field/chrom/sampleset
# internal path for calldata is chrom/calldata/field
# sampleset_calldata = sampleset_staging_dir / sset
# sampleset is needed to load species spec.
def load_single_field(zarr_path, internal_path, sset, exclude_males=False, samples=None):
      
    inz = zarr.group(is_gcloud(zarr_path), overwrite=False)
    
    oo = da.from_zarr(inz[internal_path])  
    
    if oo.ndim == 1:
        oo = oo.reshape((1, -1))
           
    return oo

In [7]:
## General function to concatenate data.
## Selected chunk size may be more appropriate for some than others.
def concatenate_along_axis(base_dir, internal_path, req_samplesets):
    
    # work out shape
    data = [load_single_field(base_dir / ss, internal_path, ss) for ss in req_samplesets]
    
    return da.concatenate(data, axis=1)

In [8]:
def is_gcloud(path):
    
    try: 
        return gcs.get_mapper(path.as_posix())
    except NameError as e:
        return path.as_posix()

In [9]:
def load_filter(chrom, filter_dir = genomic_positions_site_filter_sc_data_cloud_zarr_dir):
    gcsmap = gcs.get_mapper(filter_dir)
    genomic_positions_site_filter_data = zarr.Group(gcsmap, read_only=True)
    filter_pass = da.from_zarr(
            genomic_positions_site_filter_data[chrom]['variants/filter_pass'])
    return filter_pass

In [10]:
def load_position(chrom):
    store = gcs.get_mapper(
        f'gs://vo_afun_release/v1.0/snp_genotypes/all/sites')
    root = zarr.open(store, mode='r')
    pos = root[chrom]['variants/POS'][:]
    return pos

In [11]:
def read_in_genotypes_positions(chrom, samples_idx, samplesets, posl, posu, \
                                filter_dir = genomic_positions_site_filter_sc_data_cloud_zarr_dir):

    # load the genotypes and positions
    gt_d = concatenate_along_axis(sampleset_staging_dir, f"{chrom}/calldata/GT", samplesets)
    gt = allel.GenotypeDaskArray(gt_d)
    pos = load_position(chrom)
    
    if posu==-1:
        posu = pos.max()+1
        
    if posl==-1:
        posl = pos.min()
    
    #load the filter
    loc_filt = load_filter(chrom, filter_dir)
    
    #filter by positions
    pos_filt = (pos>=posl) & (pos<posu)
    
    #apply the filter to positions and genotypes
    gt = gt.compress((loc_filt) & (pos_filt), axis=0)
    pos = pos[(loc_filt) & (pos_filt)]
    
    #subset to desired samples 
    gt = da.take(gt, samples_idx, axis=1)
    
    return gt, pos

In [12]:
def split_to_windows(values, window_size):
    n_windows = len(values)//window_size
    vals = np.zeros(n_windows)
    for w in np.arange(n_windows):
        vals[w] = np.nanmean(values[w*window_size:(w+1)*window_size])
    return vals

In [13]:
def compute_diversity(chrom, samples_idx, posl, posu, window_size, missing_frac,\
                     samplesets):
    
    #read in genotypes
    gt, pos = read_in_genotypes_positions(chrom, samples_idx, samplesets, posl=posl, posu=posu)
    
    #count alleles 
    ac = gt.count_alleles(max_allele=3)
    
    #filters for missingness, biallelism and maf
    missing_filter = ac.sum(axis=1) >= (1-missing_frac)*2*len(samples_idx)
    
    #get filtered allele counts
    ac_f = ac.compress(missing_filter, axis=0)
    pos_f = pos[missing_filter]
    
    #compute diversity
    mpd = allel.mean_pairwise_difference(ac_f)
    
    #split to windows
    windowed_mpd = split_to_windows(mpd, window_size)
    window_centers = split_to_windows(pos_f, window_size)
    
    #get starting position new reading frame
    end_pos = pos_f[len(windowed_mpd)*window_size]
    
    return windowed_mpd, window_centers, end_pos

In [14]:
def loop_through_reading_frames(chrom, samples_idx, samplesets, outdir, cohortname,\
                                reading_frame_size, window_size, missing_frac):
    
    chrom_size = load_position(chrom).max()
    
    pi, window_centers, rf_start = compute_diversity(chrom, samples_idx, 0, reading_frame_size, window_size=window_size, \
                                    missing_frac=missing_frac, samplesets=samplesets)
    rf_end = reading_frame_size
    while rf_end < chrom_size:
        rf_end = rf_start + reading_frame_size
        pi_rf, window_rf, rf_start = compute_diversity(chrom, samples_idx, rf_start, rf_end, window_size=window_size, \
                                    missing_frac=missing_frac, samplesets=samplesets)
        pi = np.concatenate((pi, pi_rf))
        window_centers = np.concatenate((window_centers, window_rf))
        np.save(f'{outdir}/{chrom}_{cohortname}_pi.npy', pi)
        np.save(f'{outdir}/{chrom}_{cohortname}_windows.npy', window_centers)
        

In [21]:
def compute_pi_all_cohorts(chrom, cohorts, meta, outdir, samplesets=samplesets, \
                          reading_frame_size=10_000_000, window_size=20_000, missing_frac=0.05):
    
    for cohort in cohorts:
        samples_idx = meta.loc[(meta.geographic_cohort==cohort) & (meta.median_coverage>=20)].index
        loop_through_reading_frames(chrom, samples_idx, samplesets, outdir, cohort,
                                    reading_frame_size, window_size, missing_frac)
        print(f"On chromosome {chrom} cohort {cohort} done")
        
    

In [20]:
meta.columns

Index(['sample_id', 'geographic_cohort', 'geographic_cohort_colour',
       'geographic_cohort_shape', 'PCA_cohort', 'PCA_cohort_colour',
       'mitochondrial_id', 'karyotype_3La', 'karyotype_3Ra', 'karyotype_3Rb',
       'karyotype_2Ra', 'karyotype_2Rh', 'median_coverage'],
      dtype='object')

### Set up dask cluster

In [16]:
gateway = Gateway()
gateway.list_clusters()

[]

In [17]:
#gateway = Gateway()
conda_prefix = os.environ["CONDA_PREFIX"]
current_environment = 'global/'+conda_prefix.split('/')[5]
cluster = gateway.new_cluster(
    profile='standard', 
    conda_environment = current_environment,
)
cluster

In [18]:
client=cluster.get_client()

In [19]:
cluster.scale(50)

In [23]:
cohorts = meta.geographic_cohort.unique()
cohorts

array(['Ghana_Northern-Region', 'Gabon_Haut-Ogooue', 'CAR_Ombella-MPoko',
       'Cameroon_Adamawa', 'Ghana_Ashanti-Region',
       'Malawi_Southern-Region', 'Mozambique_Maputo',
       'Uganda_Eastern-Region', 'Benin_Atlantique-Dept', 'DRC_Kinshasa',
       'Nigeria_Ogun-State', 'Zambia_Eastern-Prov', 'Kenya_Nyanza-Prov',
       'Kenya_Western-Prov', 'Tanzania_Morogoro-Region', 'DRC_Haut-Uele',
       'Mozambique_Cabo-Delgado'], dtype=object)

In [24]:
compute_pi_all_cohorts('X', cohorts, meta, 'diversity/20kb_hicov/')

On chromosome X cohort Ghana_Northern-Region done
On chromosome X cohort Gabon_Haut-Ogooue done
On chromosome X cohort CAR_Ombella-MPoko done
On chromosome X cohort Cameroon_Adamawa done
On chromosome X cohort Ghana_Ashanti-Region done
On chromosome X cohort Malawi_Southern-Region done
On chromosome X cohort Mozambique_Maputo done
On chromosome X cohort Uganda_Eastern-Region done
On chromosome X cohort Benin_Atlantique-Dept done
On chromosome X cohort DRC_Kinshasa done
On chromosome X cohort Nigeria_Ogun-State done
On chromosome X cohort Zambia_Eastern-Prov done
On chromosome X cohort Kenya_Nyanza-Prov done
On chromosome X cohort Kenya_Western-Prov done
On chromosome X cohort Tanzania_Morogoro-Region done
On chromosome X cohort DRC_Haut-Uele done
On chromosome X cohort Mozambique_Cabo-Delgado done


In [25]:
compute_pi_all_cohorts('3RL', cohorts, meta, 'diversity/20kb_hicov/')

On chromosome 3RL cohort Ghana_Northern-Region done


2023-07-04 08:21:43,718 - distributed.client - WARNING - Couldn't gather 29 keys, rescheduling {"('and_-24c94e141ddbb630636de3ee2714843f', 174)": ('tls://10.97.176.10:33903',), "('and_-24c94e141ddbb630636de3ee2714843f', 29)": ('tls://10.97.176.2:34217',), "('and_-24c94e141ddbb630636de3ee2714843f', 241)": ('tls://10.97.176.10:33903',), "('and_-24c94e141ddbb630636de3ee2714843f', 58)": ('tls://10.97.176.7:38755',), "('and_-24c94e141ddbb630636de3ee2714843f', 109)": ('tls://10.97.176.10:33903',), "('and_-24c94e141ddbb630636de3ee2714843f', 220)": ('tls://10.97.176.7:38755',), "('and_-24c94e141ddbb630636de3ee2714843f', 5)": ('tls://10.97.176.7:38755',), "('and_-24c94e141ddbb630636de3ee2714843f', 56)": ('tls://10.97.176.7:38755',), "('and_-24c94e141ddbb630636de3ee2714843f', 27)": ('tls://10.97.176.2:34217',), "('and_-24c94e141ddbb630636de3ee2714843f', 271)": ('tls://10.97.176.10:33903',), "('and_-24c94e141ddbb630636de3ee2714843f', 261)": ('tls://10.97.176.7:38755',), "('and_-24c94e141ddbb63063

On chromosome 3RL cohort Gabon_Haut-Ogooue done


2023-07-04 08:22:35,820 - distributed.client - WARNING - Couldn't gather 6 keys, rescheduling {"('and_-ea046b94eeaba08ab4cf803d142727a7', 157)": (), "('and_-ea046b94eeaba08ab4cf803d142727a7', 171)": (), "('and_-ea046b94eeaba08ab4cf803d142727a7', 129)": (), "('and_-ea046b94eeaba08ab4cf803d142727a7', 114)": (), "('and_-ea046b94eeaba08ab4cf803d142727a7', 54)": (), "('and_-ea046b94eeaba08ab4cf803d142727a7', 52)": ()}


On chromosome 3RL cohort CAR_Ombella-MPoko done
On chromosome 3RL cohort Cameroon_Adamawa done
On chromosome 3RL cohort Ghana_Ashanti-Region done
On chromosome 3RL cohort Malawi_Southern-Region done
On chromosome 3RL cohort Mozambique_Maputo done
On chromosome 3RL cohort Uganda_Eastern-Region done
On chromosome 3RL cohort Benin_Atlantique-Dept done
On chromosome 3RL cohort DRC_Kinshasa done
On chromosome 3RL cohort Nigeria_Ogun-State done
On chromosome 3RL cohort Zambia_Eastern-Prov done
On chromosome 3RL cohort Kenya_Nyanza-Prov done
On chromosome 3RL cohort Kenya_Western-Prov done
On chromosome 3RL cohort Tanzania_Morogoro-Region done
On chromosome 3RL cohort DRC_Haut-Uele done
On chromosome 3RL cohort Mozambique_Cabo-Delgado done


In [26]:
compute_pi_all_cohorts('2RL', cohorts, meta, 'diversity/20kb_hicov/')

On chromosome 2RL cohort Ghana_Northern-Region done
On chromosome 2RL cohort Gabon_Haut-Ogooue done
On chromosome 2RL cohort CAR_Ombella-MPoko done
On chromosome 2RL cohort Cameroon_Adamawa done
On chromosome 2RL cohort Ghana_Ashanti-Region done
On chromosome 2RL cohort Malawi_Southern-Region done
On chromosome 2RL cohort Mozambique_Maputo done
On chromosome 2RL cohort Uganda_Eastern-Region done
On chromosome 2RL cohort Benin_Atlantique-Dept done
On chromosome 2RL cohort DRC_Kinshasa done
On chromosome 2RL cohort Nigeria_Ogun-State done
On chromosome 2RL cohort Zambia_Eastern-Prov done
On chromosome 2RL cohort Kenya_Nyanza-Prov done
On chromosome 2RL cohort Kenya_Western-Prov done
On chromosome 2RL cohort Tanzania_Morogoro-Region done
On chromosome 2RL cohort DRC_Haut-Uele done
On chromosome 2RL cohort Mozambique_Cabo-Delgado done


In [27]:
cluster.shutdown()

In [16]:
for report in gateway.list_clusters():
    gateway.connect(report.name).shutdown()